In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

from setting import *

In [ ]:
target_x_sample = pd.read_table(
    '../output/target_x_sample.tsv',
    index_col=0,
)

In [ ]:
gene_x_sample = pd.read_table(
    '../output/gene_x_sample__processed.tsv',
    index_col=0,
)

In [ ]:
gene_set_file_paths = tuple('../data/gene_set/{}'.format(file_name) for file_name in os.listdir('../data/gene_set'))

gene_set_file_paths

In [ ]:
gene_sets = ccal.read_gmts(gene_set_file_paths)

In [ ]:
from scipy.stats import pearsonr


def function(
    gene_x_sample_row,
    target,
):
   
    return pearsonr(
        gene_x_sample_row,
        target,
    )[0]

In [ ]:
for target, target_values in target_x_sample.iterrows():

    target_values = target_values[target_values != -1]
    
    output_directory_path = '../output/gsea/{}'.format(target)

    ccal.establish_path(
        output_directory_path,
        'directory',
    )
    
    gene_x_target_sample = gene_x_sample[target_values.index]
    
    gene_x_target_sample = pd.DataFrame(
        ccal.normalize_nd_array(
            gene_x_target_sample.values,
            0,
            '-0-',
            raise_for_bad_value=False,
        ),
        index=gene_x_target_sample.index,
        columns=gene_x_target_sample.columns,
    )
    
    for gene_set in (
        'Normal Stem Cell',
        'Cancer Stem Cell',
        'HALLMARK_WNT_BETA_CATENIN_SIGNALING',
    ):
        
        score, p_value = ccal.gsea(
            gene_x_target_sample,
            target_values,
            gene_sets.loc[gene_set],
            function,
            title='{}: {}'.format(
                target,
                gene_set,
            ),
            gene_score_name='Correlation',
            html_file_path='{}/{}.html'.format(
                output_directory_path,
                gene_set,
            ),
        )